# Fitting a Galprop Model to the Full Sky. 

We reproduce the global fitting analysis of http://arxiv.org/abs/1603.06584.  The fit is done in three stages using output files from Galprop.  First we fit the high latitude sky, then we fit the outer galaxy, and finally, the inner Galaxy.  XCO is allowed to float freely in this case.  

The output format is a single HDF5 file, which will be useful in later analysis. If one does not wish to fit a galprop model, but instead just use the output, see the tutorial, using Galprop output. 


This analysis assumes that the galprop output has 9 rings for each gas component.  These input gas maps to galprop can be found in the support_files folder of the GammaLike github repo.

In [ ]:
import sys
sys.path.append('/data/GammaLike/')
from GammaLike import Analysis

basepath = '/data/GammaLike/testing/'
tag = 'P8R2_CLEAN_V6_calore' 
A = Analysis.Load(basepath+tag+'.GLanalysis')



A.AddIsotropicTemplate(
        isofile='/data/GCE_sys/IGRB_ackerman_2014_modA.dat', # See the appendix at the bottom of the file. 
        fixNorm=False, fixSpectrum=False) 

A.AddPointSourceTemplate(fixNorm=True, # Fix the template normalization in all bins.
                         pscmap=A.basepath + '/PSC_' + A.tag + '_fgl3_with_ext.npy') # This was generated in the first tutorial

A.CalculatePixelWeights(diffuse_model=A.basepath+'/'+'fermi_diffuse_'+A.tag+'.npy',
                        psc_model=A.basepath + '/PSC_' + A.tag + '_fgl3_with_ext.npy',
                        alpha_psc=5., f_psc=0.1) # see 1409.0042 Eq.(2.6)

A.AddFermiBubbleTemplate(template_file='./bubble_templates_diskcut30.0.fits',
                         spec_file='./reduced_bubble_spec_apj_793_64.dat', fixSpectrum=True, fixNorm=False)



In [ ]:
def GenDiffuse(self, basedir='/data/galprop2/output/',
               tag='NSPEB_no_secondary_HI_H2', verbosity=0, multiplier=1., nrings=9.,
                E_subsample=1, fixSpectrum=True, fix_xco=False):
        """
        This method takes a base analysis prefix, along with an X_CO profile and generates the combined diffuse template,
        or components of the diffuse template.

        :param basedir: Base directory to read from
        :param tag: Tag for the galprop file.  This is the part between '_54_' and '.gz'.
        :param verbosity: 0 is quiet, >1 prints status.
        """

        if verbosity>0:
            print 'Loading FITS'

        energies = pyfits.open(basedir+'/bremss_healpix_54_'+tag+'.gz')[2].data.field(0)
        comps, comps_new = {}, {}

        def ReadFits(fname, length):
            d = pyfits.open(fname)[1].data
            return np.array([d.field(i) for i in range(length)])
        
        # Add up the HI and HII contributions into a single template since nothing there is varying.
        pi0HIHII = np.zeros((len(energies), 12*self.nside**2))
        bremHIHII = np.zeros((len(energies), 12*self.nside**2))
        
        for i_ring in range(1,nrings+1):
            print "Adding HI/HII ring", i_ring
            bremHIHII += ReadFits(basedir+'/bremss_HIR_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
            bremHIHII += ReadFits(basedir+'/bremss_HII_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
            pi0HIHII += ReadFits(basedir+'/pi0_decay_HIR_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
            pi0HIHII += ReadFits(basedir+'/pi0_decay_HII_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
            if fix_xco:
                bremHIHII += ReadFits(basedir+'/bremss_H2R_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
                pi0HIHII += ReadFits(basedir+'/pi0_decay_H2R_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))


        comps['pi0HIHII'] = pi0HIHII + 1.25*bremHIHII
        # comps['pi0HIHII'] = pi0HIHII
        # comps['bremHIHII'] = bremHIHII
        comps_new['pi0HIHII'] =  np.zeros((self.n_bins, 12*self.nside**2))
        # comps_new['bremHIHII'] =  np.zeros((self.n_bins, 12*self.nside**2))
        

        for i_ring in range(1,nrings+1):
            if not fix_xco:
                print "Adding H2 ring", i_ring
                brem = ReadFits(basedir+'/bremss_H2R_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
                pi = ReadFits(basedir+'/pi0_decay_H2R_ring_'+str(i_ring)+'_healpix_54_'+tag+'.gz', len(energies))
                comps['pi0_H2_'+str(i_ring)] = pi + 1.25*brem

                # comps['pi0_H2_'+str(i_ring)] = pi 
                # comps['brem_H2_'+str(i_ring)]= brem
                comps_new['pi0_H2_'+str(i_ring)] =  np.zeros((self.n_bins, 12*self.nside**2))
                # comps_new['brem_H2_'+str(i_ring)] = np.zeros((self.n_bins, 12*self.nside**2))
            
        comps['ics'] = np.zeros((len(energies), 12*self.nside**2))
        comps_new['ics'] = np.zeros((self.n_bins, 12*self.nside**2))
        for i_ics in range(1,4):
            print "Adding ics", i_ics
            comps['ics'] += ReadFits(basedir+'/ics_isotropic_comp_'+str(i_ics)+'_healpix_54_'+tag+'.gz', len(energies))
        
        nside_in = np.sqrt(comps['pi0HIHII'].shape[1]/12)
        
        #---------------------------------------------------------------------------------
        # Now we integrate each model over the energy bins...
        #
        # Multiprocessing for speed. There is an async callback which applies each result to
        # the arrays.  Not sure why RunAsync needs new thread pool for each component, but this
        # works and decreases memory footprint.
        def callback(result):
            idx, comp, dat = result
            comps_new[comp][idx] = dat

        def RunAsync(component):
            p = mp.Pool(mp.cpu_count())
            for i_E in range(self.n_bins):
                p.apply_async(Tools.AsyncInterpolateHealpix,
                              [comps[component], energies, self.bin_edges[i_E], self.bin_edges[i_E+1],
                               i_E, component, E_subsample, self.nside],
                              callback=callback)
            p.close()
            p.join()

        # For each component, run the async sampling/sizing.
        for key in comps:
            if verbosity>0:
                print 'Integrating and Resampling', key, 'templates...'
                sys.stdout.flush()
            for i_E in range(self.n_bins):
                comps_new[key][i_E] = Tools.InterpolateHealpix(comps[key], energies,  
                    self.bin_edges[i_E], self.bin_edges[i_E+1], E_bins=E_subsample, nside_out=self.nside)
            # Parallel version. (very memory hungry)
            #RunAsync(key)


        #---------------------------------------------------------------------------------
        # Now we just need to add the templates to the active template stack
        print 'Adding Templates to stack'
        
        self.AddTemplate(name='pi0HIHII', healpixCube=comps_new['pi0HIHII'].clip(0), fixSpectrum=fixSpectrum, fixNorm=False,
                           value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)
        # self.AddTemplate(name='bremHIHII', healpixCube=comps_new['bremHIHII'], fixSpectrum=fixSpectrum, fixNorm=False,
                           # value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)
        
        for i_ring in range(1,nrings+1):
            if not fix_xco:
                self.AddTemplate(name='pi0_H2_'+str(i_ring), healpixCube=comps_new['pi0_H2_'+str(i_ring)].clip(0), fixSpectrum=fixSpectrum, fixNorm=False,
                               value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)
                
                # self.AddTemplate(name='brem_H2_'+str(i_ring), healpixCube=comps_new['brem_H2_'+str(i_ring)], fixSpectrum=fixSpectrum, fixNorm=False,
                #                value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)
            
    #         for i_ics in range(1,4):
    #             self.AddTemplate(name='ics_'+str(i_ics), healpixCube=comps_new['ics_'+str(i_ics)], fixSpectrum=fixSpectrum, fixNorm=False,
    #                            value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)
        self.AddTemplate(name='ics', healpixCube=comps_new['ics'].clip(0), fixSpectrum=fixSpectrum, fixNorm=False,
                       value=1., ApplyIRF=True,noPSF=True, sourceClass='GEN', limits=[None, None], multiplier=multiplier)

        return self
